# LlamaIndex: Starter Tutorial (Using OpenAI)

- Starter Tutorial (Using OpenAI)<br>
  https://docs.llamaindex.ai/en/stable/getting_started/starter_example/

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Set OpenAI API key
LlamaIndex uses **OpenAI (gpt-3.5-turbo)** by default.<br>
```
# MacOS/Linux
export OPENAI_API_KEY=XXXXX

# Windows
set OPENAI_API_KEY=XXXXX
```

## Basic Agent Example

In [2]:
import asyncio
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b

# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    name="Agent",
    description="Useful for multiplying two numbers",
    tools=[multiply],
    llm=OpenAI(model="gpt-4o-mini"),
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)

# async def main():
#     # Run the agent
#     response = await agent.run("What is 1234 * 4567?")
#     print(str(response))

# # Run the agent
# if __name__ == "__main__":
#     asyncio.run(main())

In [3]:
# Run the agent
response = await agent.run("What is 1234 * 4567?")

In [4]:
print(response)

The result of \( 1234 \times 4567 \) is \( 5,635,678 \).


## Adding Chat History

In [5]:
# from llama_index.core.workflow import Context

# # create context
# ctx = Context(agent)

# # run agent with context
# response = await agent.run("My name is Logan", ctx=ctx)
# response = await agent.run("What is my name?", ctx=ctx)

In [6]:
# ERROR: AttributeError: 'FunctionAgent' object has no attribute '_get_steps'

## Adding Chat History by (AgentWorkflow Basic Introduction)
- https://docs.llamaindex.ai/en/stable/examples/agent/agent_workflow_basic/

In [7]:
# %pip install llama-index
# %pip install tavily-python

In [8]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")

In [9]:
# Creating a tool
import os
from tavily import AsyncTavilyClient

tavily_api_key = os.getenv("TAVILY_API_KEY")

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    return str(await client.search(query))

In [10]:
# Creating an AgentWorkflow that uses the tool
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web for information.",
)

In [11]:
# Running the Agent
# response = await workflow.run(user_msg="What is the weather in San Francisco?")
response = await workflow.run(user_msg="What is the weather in Prague?")
print(str(response))

The current weather in Prague is as follows:

- **Temperature**: 8.0°C (46.4°F)
- **Condition**: Light rain
- **Humidity**: 81%
- **Wind**: 6.0 mph (9.7 kph) from the WNW
- **Pressure**: 1018 mb
- **Visibility**: 5.0 km

It is currently daytime in Prague, and the weather feels like 6.3°C (43.3°F). You can expect light rain, so it might be a good idea to carry an umbrella if you're going out.

For more details, you can check the full weather report [here](https://www.weatherapi.com/).


In [12]:
# Maintaining State
from llama_index.core.workflow import Context

ctx = Context(workflow)

response = await workflow.run(
    user_msg="My name is Logan, nice to meet you!", ctx=ctx
)
print(str(response))

Nice to meet you, Logan! How can I assist you today?


In [13]:
response = await workflow.run(user_msg="What is my name?", ctx=ctx)
print(str(response))

Your name is Logan.


## Adding RAG Capabilities

Example data
```
mkdir data
wget https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt -O data/paul_graham_essay.txt
```
-  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt

In [14]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
import asyncio
import os

# Create a RAG tool using LlamaIndex
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b

async def search_documents(query: str) -> str:
    """Useful for answering natural language questions about an personal essay written by Paul Graham."""
    response = await query_engine.aquery(query)
    return str(response)

# Create an enhanced workflow with both tools
agent = FunctionAgent(
    name="Agent",
    description="Useful for multiplying two numbers and searching through documents to answer questions.",
    tools=[multiply, search_documents],
    llm=OpenAI(model="gpt-4o-mini"),
    system_prompt="""You are a helpful assistant that can perform calculations
    and search through documents to answer questions.""",
)

# # Now we can ask questions about the documents or do calculations
# async def main():
#     response = await agent.run(
#         "What did the author do in college? Also, what's 7 * 8?"
#     )
#     print(response)

# # Run the agent
# if __name__ == "__main__":
#     asyncio.run(main())

In [15]:
response = await agent.run("What did the author do in college? Also, what's 7 * 8?")
print(response)

In college, the author initially planned to study philosophy but found the courses boring. Consequently, the author switched to studying AI, influenced by a novel called "The Moon is a Harsh Mistress" by Heinlein and a PBS documentary featuring Terry Winograd using SHRDLU.

As for the multiplication, \(7 \times 8 = 56\).


## Storing the RAG Index

In [16]:
# Save the index
index.storage_context.persist("storage")

# Later, load the index
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="storage")
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine()